# OT for the Kyle model with insider activism

In [ ]:
from __future__ import division

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy as scp
import pylab as pyl

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Definition of the parameters

In [ ]:
def solve_sinkhorn(M, N, m, epsilon, params, debug=False):
    # Define grids for variables y,z,xi
    grid_y  = np.linspace(-m,m,N)
    grid_z  = np.linspace(-m,m,N)
    grid_xi = np.linspace(0,1,M)
    [Y,Z,Xi] = np.meshgrid(grid_y,grid_z,grid_xi)
    dimensions = (N, N, M) 
    
    # Increments
    dy  = 2*m/N
    dz  = dy
    dxi = 1.0/M

    # Copy hyperparameters
    sigma = params['sigma']
    T     = params['T']
    sigma_beta = params['sigma_beta']
    m_beta     = params['m_beta']
    delta      = params['delta']
    x_star     = params['x_star']
    
    # Target measures
    # - Law of Y
    v_eff = sigma*sigma*T + sigma_beta*sigma_beta
    a = dy*np.exp( -0.5*grid_y*grid_y/v_eff)/np.sqrt(2*np.pi*v_eff)
    # - Law of (Z-beta, Xi)
    v_eff = sigma*sigma*T + sigma_beta*sigma_beta
    b = dz*np.exp( -m_beta*grid_y - 0.5*grid_y*grid_y/v_eff)/np.sqrt(2*np.pi*v_eff)
    [b1, b2] = np.meshgrid(b, dxi* np.ones_like(grid_xi))
    b = b1*b2 # Product measure
    b = b.T   # For some reason, dim order is correct this way
    a = a/np.sum(a)
    b = b/np.sum(b)
    if debug:
        print( "Grid for measure a:", a.shape )
        print( "Grid for measure b:", b.shape )

    # Kernel
    surplus = (Y-Z)*Xi + delta*np.fmax(Y-Z-x_star, 0)
    K1 = np.exp( surplus/epsilon )
    if debug:
        print( "Grid for kernel:", K1.shape )

    #
    # Launch Sinkhorn
    v = np.ones_like(b)
    K  = lambda x : np.tensordot( K1, x, axes=[[1,2], [0,1]])
    KT = lambda x : np.tensordot( K1, x, axes=[[0], [0]])

    count = 1000
    errors = []
    for i in range(count):
        u = a / K(v)
        v = b / KT(u)
        e = np.linalg.norm( u[None,:]*K(v) - a )
        errors.append( e )
    #
    # Plot
    errors_sinkhorn = errors
    plt.yscale('log')
    plt.plot(errors)
    plt.ylabel('Error in log-scale')
    plt.xlabel('Number of iterations')
    #
    return u, K1, v

def plot_level_set(plan, M, N, m):
    print("Total mass: ", np.sum(plan))
    # Define grids for variables y,z,xi
    grid_y  = np.linspace(-m,m,N)
    grid_z  = np.linspace(-m,m,N)
    grid_xi = np.linspace(0,1,M)
    # Set colors
    plt.figure(figsize=(10, 8), dpi=80)
    cmap = plt.get_cmap('viridis',N)
    norm = matplotlib.colors.Normalize(vmin=0,vmax=1)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    #
    xi_old  = np.zeros_like( grid_z )
    for i in range(N):
        # Slice for fixed y
        # Note: Slice = Conditionnal distribution (Z,Xi | Y)
        plan_slice = plan[i,:,:]
        plan_slice = plan_slice/np.sum(plan_slice)
        # Compute mean of Xi conditionally to Z
        xi = np.dot( plan_slice, grid_xi )/np.sum(plan_slice, axis=1)
        plt.fill_between(x=grid_z, y1=xi_old, y2=xi, color=cmap( norm(i/N) ) )
        xi_old = xi
    plt.xlabel('Z axis')
    plt.ylabel('Xi axis')
    plt.colorbar(sm, ticks=np.linspace(-3,3,21), 
                 boundaries=np.arange(-3,3,0.1))
    plt.show()

def plot_level_set2(plan, M, N, m):
    print("Total mass: ", np.sum(plan))
    # Define grids for variables y,z,xi
    grid_y  = np.linspace(-m,m,N)
    grid_z  = np.linspace(-m,m,N)
    grid_xi = np.linspace(0,1,M)
    # Set colors
    plt.figure(figsize=(10, 8), dpi=80)
    cmap = plt.get_cmap('viridis',N)
    norm = matplotlib.colors.Normalize(vmin=0,vmax=1)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    #
    xi_old  = np.zeros_like( grid_z )
    for i in range(N):
        # Slice for fixed y
        # Note: Slice = Conditionnal distribution (Z,Xi | Y)
        plan_slice = plan[i,:,:]
        plan_slice = plan_slice/np.sum(plan_slice)
        # Compute mean of Xi conditionally to Z
        z = grid_z
        xi = np.dot( plan_slice, grid_xi )/np.sum(plan_slice, axis=1)
        plt.plot( grid_z, xi )
    plt.xlabel('Z axis')
    plt.ylabel('Xi axis')
    plt.show()

In [ ]:
# Hyperparameters
hyperparameters = {
    'sigma'     : 1.0,
    'T'         : 1.0,
    'm_beta'    : 0.0,
    'sigma_beta': 1.0,
    'delta'     : 1.0,
    'x_star'    : 0.5
}

# Sampling parameters in grid
N = 100
M = 80
m = 4

# Sinkorn regularization parameter
epsilon = 0.05

# Solve Sinkhorn
potential_a, K1, potential_b = solve_sinkhorn(M,N,m,epsilon=epsilon, params=hyperparameters)
# Form transport plan
plan = potential_a[:, None, None]*K1*potential_b[None, :, :]

In [ ]:
plot_level_set(plan, M, N, m)
plot_level_set2(plan, M, N, m)

In [ ]:
def compute_I(plan, M, N, m, debug=False):
    if debug:
        print("Total mass: ", np.sum(plan))
    # Define grids for variables y,z,xi
    grid_y  = np.linspace(-m,m,N)
    grid_z  = np.linspace(-m,m,N)
    grid_xi = np.linspace(0,1,M)

    # Height is conditional mean Y | (Z,Xi)
    conditional_mass = np.tensordot( np.ones_like(grid_y), plan, axes=[ [0], [0]] )
    height = plan/conditional_mass[None, :,:]
    height = np.tensordot( grid_y, height, axes=[ [0], [0]] )
    
    return grid_z, grid_xi, height.T

# II. Plot in 3D in plotly by layering Y

In [ ]:
# Solve Sinkhorn
potential_a, K1, potential_b = solve_sinkhorn(M,N,m,epsilon=epsilon, params=hyperparameters)
# Form transport plan
plan = potential_a[:, None, None]*K1*potential_b[None, :, :]

In [ ]:
import plotly.graph_objects as go

# Define grids for variables y,z,xi
grid_y  = np.linspace(-m,m,N)
grid_z  = np.linspace(-m,m,N)
grid_xi = np.linspace(0,1,M)

#
height = np.zeros( (N,N) )
for i in range(N):
    # Slice for fixed y
    # Note: Slice = Conditionnal distribution (Z,Xi | Y)
    plan_slice = plan[i,:,:]
    plan_slice = plan_slice/np.sum(plan_slice)
    # Compute mean of Xi conditionally to Z
    z = grid_z
    xi = np.dot( plan_slice, grid_xi )/np.sum(plan_slice, axis=1)
    height[i,:] = xi

fig = go.Figure( data=[
    go.Surface(x=grid_y, y=grid_z, z=height)
] )
fig.update_layout(title_text="Y = I(Z,Xi)",
                  scene = dict(
                     xaxis_title="Y",
                     yaxis_title="Z",
                     zaxis_title="Xi",
                     xaxis = dict(nticks=10, range=[-m,m],),
                     yaxis = dict(nticks=10, range=[-m,m],),
                     zaxis = dict(nticks=4, range=[0,1],),
                  )
                 )
fig.show()

# III. Plot in 3D using conditioning for computing I

In [ ]:
# Hyperparameters
hyperparameters = {
    'sigma'     : 1.0,
    'T'         : 1.0,
    'm_beta'    : 0.0,
    'sigma_beta': 1.0,
    'delta'     : 1.0,
    'x_star'    : 0.5
}


# Sampling parameters in grid
N = 100
M = 80
m = 4

# Sinkorn regularization parameter
epsilon = 0.05

# Loop over hyperparameters
Is = []
for param in [-1.0, 0.0, 1.0]:
    params=dict(hyperparameters)
    params['m_beta'] = param
    # Solve Sinkhorn
    potential_a, K1, potential_b = solve_sinkhorn(M,N,m,epsilon=epsilon, params=params)
    # Form transport plan
    plan = potential_a[:, None, None]*K1*potential_b[None, :, :]
    # Compute I
    grid_z, grid_y, height = compute_I( plan, M, N, m)
    Is.append( height )


import plotly.graph_objects as go

data = []
for I in Is:
    data.append( go.Surface(x=grid_z, y=grid_xi, z=I) )
fig = go.Figure( data=data )
fig.update_layout(title_text="Y = I(Z,Xi)",
                  scene = {
                     'xaxis_title': 'Z',
                     'yaxis_title': 'Xi',
                     'zaxis_title': 'Y',
                     'xaxis': { 'nticks':10, 'range': [-m,m] },
                     'yaxis': { 'nticks':10, 'range': [0,1]  },
                     'zaxis': { 'nticks':10, 'range': [-m,m] },
                  })
fig.write_image("multiple_m.png")
fig.show()


# IV. Probability of taking a role in governance

In [ ]:
# Hyperparameters
hyperparameters = {
    'sigma'     : 1.0,
    'T'         : 1.0,
    'm_beta'    : -1.0,
    'sigma_beta': 1.0,
    'delta'     : 1.0,
    'x_star'    : 0.5
}


# Sampling parameters in grid
N = 100
M = 80
m = 4

# Sinkorn regularization parameter
epsilon = 0.05

# Loop over hyperparameters
probabilities = []
for param in np.linspace( 0.1, 2, 20):
    params=dict(hyperparameters)
    params['sigma'] = param
    # Solve Sinkhorn
    potential_a, K1, potential_b = solve_sinkhorn(M,N,m,epsilon=epsilon, params=params)
    # Form transport plan
    plan = potential_a[:, None, None]*K1*potential_b[None, :, :]
    # Define grids for variables y,z,xi
    grid_y  = np.linspace(-m,m,N)
    grid_z  = np.linspace(-m,m,N)
    grid_xi = np.linspace(0,1,M)
    # Normalize for safety
    plan = plan/np.sum(plan)
    # Law of (Y,Z)
    law_YZ = np.sum( plan, axis=2 )
    #
    y, z = np.meshgrid( grid_y, grid_z )
    indices = np.where( y-z>hyperparameters['x_star'])
    p = np.sum( law_YZ[indices])
    probabilities.append( p )

In [ ]:
# Plot
plt.figure(figsize=(10, 8), dpi=80)
plt.plot( np.linspace( 0, 1, 20), probabilities )
plt.title( '$\mathbb{P}( Y-Z \geq x^*)$ as a function of $\sigma$.')
plt.xlabel('$\sigma$')
plt.ylabel('probability')
plt.ylim( [0,1] )
plt.savefig('proba.png')
plt.show()